# Basic examples
This notebook outlines the construction of Hamiltonians which are not inherently useful
on their own, but are of a similar form to what we expect for the most basic
simulations where a quantum computer may be useful. This version is a working version and
will likely be updated later to include more application relevant examples.  The dynamic
simulation of these Hamiltonians can be thought of as a necessary but not sufficient
condition for demonstrating the viability of a quantum computer as a useful tool for 
quanutm dynamic simulation.

Note that running this entire notebook should take on the order of an hour.
To see smaller cases, simply reduce the sizes of the lattices for the various Hamiltonians.

## Basic Hamiltonian
In this section we consider the time independent transverse field Ising Hamiltonians represented by graphs
with a lattice structure. There are more complicated models which would likely
be interesting to simulate, but as an initial pass we start by considering two models
* 32x32 Triangular Lattice (1024 spins, 2 dimensions) with antiferromagnetic unit couplings
* 12x12x12 Cubic Lattice (1728 spins, 3 dimensions) with random unit couplings
The transverse field Ising Hamiltonian for a lattice graph, $G = (V,E)$ is as follows:

\begin{equation}
H = \sum_{i \in V} \Gamma_{i}\sigma_{i}^{x} + \sum_{i \in V} h_{i} \sigma_{i}^{z} + \sum_{(i,j) \in E} J_{i,j} \sigma_{i}^{z} \sigma_{j}^{z}
\end{equation}
Note that in the instances presented here, we do not consider local longitudinal field terms ($\boldsymbol{h}=0$), though this may not always be the case.
We also do not consider in this example the state preparation circuit that may be required as that is heavily application dependent.
For the purposes of these initial experiments, we can assume that the initial state is an eigenstate of either the $X$ or $Z$ basis since the preparation circuits for these have $O(1)$ depth.

To be clear, the dynamic simulations mean simulating the Schrodinger Equation
\begin{equation}
i \hbar \frac{\partial}{\partial t}|\psi(t)\rangle = H |\psi(t)\rangle
\end{equation}
with solution
\begin{equation}
|\psi(t)\rangle = e^{-\frac{i}{\hbar}H t}|\psi(0)\rangle
\end{equation}

A subgraph of each lattice is plotted below to clarify the graph structure to clarify the graph structure.

In [ ]:
import os
import time
import json
import random
import numpy as np
import networkx as nx
from networkx import grid_graph
from openfermion import count_qubits
from networkx.classes.graph import Graph
from openfermion.circuits import error_bound
from openfermion.circuits import trotter_steps_required
from networkx.generators.lattice import hexagonal_lattice_graph
from openfermion.circuits.trotter_exp_to_qgates import trotterize_exp_qubop_to_qasm
from qca.utils.utils import count_gates, get_T_depth_wire, get_T_depth, plot_histogram, gen_resource_estimate, re_as_json
from qca.utils.hamiltonian_utils import (nx_triangle_lattice, flatten_nx_graph,
                                         generate_square_hamiltonian, pyliqtr_hamiltonian_to_openfermion_qubit_operator,
                                         assign_directional_triangular_labels, generate_triangle_hamiltonian,
                                         assign_hexagon_labels)


In [ ]:
import cirq
from cirq.contrib import qasm_import
from pyLIQTR.circuits.qsp import generate_QSP_circuit
from pyLIQTR.utils.qsp_helpers import print_to_openqasm
from pyLIQTR.utils.Hamiltonian import Hamiltonian as pyH
from pyLIQTR.utils.qsp_helpers import circuit_decompose_once
from pyLIQTR.utils.utils import open_fermion_to_qasm, count_T_gates
from pyLIQTR.gate_decomp.cirq_transforms import clifford_plus_t_direct_transform
from pyLIQTR.phase_factors.fourier_response.fourier_response import Angler_fourier_response


In [ ]:
lattice_size = 3
graph_square = grid_graph(dim = (lattice_size, lattice_size))
graph_triangle = nx_triangle_lattice(lattice_size)
graph_cube = grid_graph(dim = (lattice_size, lattice_size, lattice_size))

In [ ]:
nx.draw(graph_square)

In [ ]:
nx.draw(graph_triangle)

In [ ]:
nx.draw(graph_cube)

Now that we have an idea of what our Hamiltonians look like, we can begin generating them in a form accepted by pyLIQTR.

In [ ]:
def estimate_qsp(pyliqtr_hamiltonian, timesteps, energy_precision, outdir, hamiltonian_name='hamiltonian', write_circuits=False,):
    timestep_of_interest=1 #for magnus like argument
    t0 = time.perf_counter()
    random.seed(0)
    np.random.seed(0)
    angles_response = Angler_fourier_response(tau=timestep_of_interest*pyliqtr_hamiltonian.alpha, eps=energy_precision, random=True, silent=True)
    angles_response.generate()
    angles = angles_response.phases

    qsp_circuit = generate_QSP_circuit(pyliqtr_hamiltonian, angles, pyliqtr_hamiltonian.problem_size)
    t1 = time.perf_counter()
    elapsed = t1 - t0
    print("  Time to generate high level QSP circuit: " + str(elapsed) + " seconds")
    
    if not os.path.exists(outdir):
        os.makedirs(outdir)
    
    subcircuit_counts = dict()
    
    
    for moment in qsp_circuit:
        for operation in moment:
            gate_type = type(operation.gate)
            if gate_type in subcircuit_counts:
                subcircuit_counts[gate_type][0] += 1
            else:
                t0 = time.perf_counter()
                decomposed_circuit = circuit_decompose_once(circuit_decompose_once(cirq.Circuit(operation)))
                t1 = time.perf_counter()
                elapsed = t1 - t0
                print("    Time to decompose high level " + str(gate_type)[8:-2] +" circuit: " + str(elapsed) + " seconds")
                
                t0 = time.perf_counter()
                cpt_circuit = clifford_plus_t_direct_transform(decomposed_circuit)
                t1 = time.perf_counter()
                elapsed = t1 - t0
                print("    Time to transform decomposed " + str(gate_type)[8:-2] + " circuit to Clifford+T: " + str(elapsed) + " seconds")
                
                circ_name = f'{outdir}{str(gate_type).split(".")[-1][:-2]}'
                if write_circuits:
                    outfile_qasm_decomposed = f'{circ_name}.decomposed.qasm'
                    outfile_qasm_cpt = f'{circ_name}.cpt.qasm'
                    with open(outfile_qasm_decomposed, 'w') as f:
                        print_to_openqasm(f, decomposed_circuit, qubits=decomposed_circuit.all_qubits())
                    
                    with open(outfile_qasm_cpt, 'w') as f:
                        print_to_openqasm(f, cpt_circuit, qubits=cpt_circuit.all_qubits())
                
                subcircuit_counts[gate_type] = [1, cpt_circuit, circ_name]
                
    total_gate_count = 0
    total_gate_depth = 0
    total_T_depth = 0
    total_T_depth_wire = 0
    total_T_count = 0
    total_clifford_count = 0
    subcircuit_re = []
    for gate in subcircuit_counts:
        circuit = subcircuit_counts[gate][1]
        circuit_name = subcircuit_counts[gate][2]
        resource_estimate = gen_resource_estimate(circuit,
                                                  circ_occurences=subcircuit_counts[gate][0]*timesteps)
        subcircuit_info = {circuit_name:resource_estimate}
        subcircuit_re.append(subcircuit_info)
        gate_count = resource_estimate['gate_count']
        gate_depth = resource_estimate['circuit_depth']
        t_depth = resource_estimate['t_depth']
        t_depth_wire = resource_estimate['max_t_depth_wire']
        t_count = resource_estimate['t_count']
        clifford_count = resource_estimate['clifford_count']
        
        total_gate_count += subcircuit_counts[gate][0] * gate_count * timesteps / timestep_of_interest
        total_gate_depth += subcircuit_counts[gate][0] * gate_depth * timesteps / timestep_of_interest
        total_T_depth += subcircuit_counts[gate][0] * t_depth * timesteps / timestep_of_interest
        total_T_depth_wire += subcircuit_counts[gate][0] * t_depth_wire * timesteps / timestep_of_interest
        total_T_count += subcircuit_counts[gate][0] * t_count * timesteps / timestep_of_interest
        total_clifford_count += subcircuit_counts[gate][0] * clifford_count * timesteps / timestep_of_interest

    outfile_data =f'{outdir}{hamiltonian_name}_high_level.json'
    total_resources = {
        'num_qubits': len(qsp_circuit.all_qubits()),
        'gate_count': total_gate_count,
        'circuit_depth': total_gate_depth,
        't_count': total_T_count,
        't_depth': total_T_depth,
        't_depth_wire': total_T_depth_wire,
        'clifford_count': total_clifford_count
    }
    re_as_json(total_resources, subcircuit_re, outfile_data)

    return qsp_circuit

In [ ]:
## initializing seed for consistent results for the cubic lattice and generating Hamiltonians
random.seed(0)
np.random.seed(0)
square_lattice_size=10
triangle_lattice_size=32
cubic_lattice_size=12
test_lattice_size=3

square_hamiltonian = generate_square_hamiltonian(square_lattice_size, dim=2)
cubic_hamiltonian = generate_square_hamiltonian(cubic_lattice_size, dim=3)
triangular_hamiltonian = generate_triangle_hamiltonian(triangle_lattice_size)

Now that we have generated our Hamiltonians, we can start generating the circuits for a preferred dynamic simulation method (e.g., QSP, Trotter 4th order, ...).  Shown below is QSP.

In [ ]:
timesteps=1000
required_precision = 1e-16

#feeding square Hamiltonian to PyLIQTR for circuit generation
H_square = pyH(square_hamiltonian[0] + square_hamiltonian[1])
H_triangle = pyH(triangular_hamiltonian[0] + triangular_hamiltonian[1])
H_cube = pyH(cubic_hamiltonian[0] + cubic_hamiltonian[1])

Now we need to extract resource estimates for each of the subcircuits of the high level circuit.  The information which we are interested in is as follows:
* Total gate count for full circuit
* Total depth for full circuit
* Decomposed QASM circuit for each subcircuit
* Clifford + T circuits for each subcircuit
* Total T gate count for each subcircuit
* Total Clifford count for each subcircuit
* T gate depth for each subcircuit

This information can be extracted by obtaining the information for each subcircuit and multiplying by the number of repetitions of the subcircuits.  In this
notebook, we show the process for both QSP and second order Suzuki-Trotter, though this process could likely be generalized to many quantum algorithms.
It should be noted that the approach shown here provides a slight over-estimation since it assumes that each of the subcircuits operate in serial.  This
assumption is valid for both QSP and second order Suzuki-Trotter since the operations which consume the bulk of the circuit volume do operate in serial
(Select and Reflect Operations and Trotter steps respectively).

In [ ]:
print('Estimating Square', flush=True)
qsp_circ_square = estimate_qsp(H_square, timesteps, required_precision, 'QSP/square_circuits/', hamiltonian_name='square')
print('Estimating Triangle', flush=True)
qsp_circ_triangle = estimate_qsp(H_triangle, timesteps, required_precision, 'QSP/triangle_circuits/', hamiltonian_name='triangle')
print('Estimating Cube', flush=True)
qsp_circ_cube = estimate_qsp(H_cube, timesteps, required_precision, 'QSP/cube_circuits/', hamiltonian_name='cube')
print('Finished estimating', flush=True)

# More complicated Hamiltonian
There are a few ways to make the simulation task more challenging and more application relevant.
The Kitaev honeycomb model is hypothesized to be a good model for the structure of many materials (see for example [here](https://www.sciencedirect.com/science/article/pii/S0370157321004051)).  This model is appealing for quantum simulation purposes because while the behavior in the infinite time regime is [understood](https://www.cambridge.org/core/books/introduction-to-topological-quantum-computation/kitaevs-honeycomb-lattice-model/85E968DD7C54F8062C4EAF5394DAAAF6), the behavior with minor perturbations to the model or the dynamics of the model are not so well [understood](https://courses.physics.illinois.edu/phys598PTD/fa2013/L26.pdf).  The Kitaev honeycomb model consists of directionally defined $XX$, $YY$, and $ZZ$ couplings on a honeycomb lattice.  These assignments are denoted in the plot below by 'X', 'Y', or 'Z' respectively.

In [ ]:
hexagon_graph = hexagonal_lattice_graph(3,3)
pos = nx.get_node_attributes(hexagon_graph, 'pos')
assign_hexagon_labels(hexagon_graph, 'X', 'Y', 'Z')
edge_labels = dict([((n1, n2), d['label']) for n1, n2, d in hexagon_graph.edges(data=True)]);
nx.draw(hexagon_graph, pos)
nx.draw_networkx_edge_labels(hexagon_graph, pos,edge_labels = edge_labels);

From this graph we can generate the Hamiltonian
\begin{equation*}
H_{\text{Kitaev}} = \sum_{(i,j) \in X \text{ edges}} J_{i,j} \sigma_i^x \sigma_j^x + \sum_{(i,j) \in Y \text{ edges}} J_{i,j} \sigma_i^y \sigma_j^y + \sum_{(i,j) \in Z \text{ edges}} J_{i,j} \sigma_i^z \sigma_j^z
\end{equation*}

In [ ]:
def nx_kitaev_terms(g:Graph, p:float) -> list:
    hamiltonian = []
    n = len(g.nodes)
    for (n1, n2, d) in g.edges(data=True):
        label = d['label']
        weight = 1 if random.random() < p else -1
        pauli_string = n * 'I'
        for i in range(len(g)):
            if i == n1 or i == n2:
                pauli_string = f'{pauli_string[:i]}{label}{pauli_string[i+1:]}'
            else:
                pass
        hamiltonian.append((pauli_string, weight))
    return hamiltonian

def generate_kitaev_hamiltonian(lattice_size:int, weight_prob:float=1):
    graph = hexagonal_lattice_graph(lattice_size,lattice_size)
    assign_hexagon_labels(graph, 'X', 'Y', 'Z')
    graph = flatten_nx_graph(graph)
    H = nx_kitaev_terms(graph, weight_prob)
    return H


In [ ]:
# lattice_size_kitaev = 3
lattice_size_kitaev = 32
kitaev_hamiltonian = generate_kitaev_hamiltonian(lattice_size_kitaev)

timesteps = 1000
required_precision = 1e-16
H_kitaev = pyH(kitaev_hamiltonian)
print('Estimating Kitaev', flush=True)
qsp_circ_kitaev = estimate_qsp(H_kitaev, timesteps, 
                               required_precision, 
                               'QSP/kitaev_circuits/',
                               hamiltonian_name='kitaev')
print('Finished Estimating', flush=True)

In [ ]:
def generate_directional_triangular_hamiltonian(lattice_size:int, weight_prob:float = 1):
    graph = nx_triangle_lattice(lattice_size)
    assign_directional_triangular_labels(graph,lattice_size)
    graph = flatten_nx_graph(graph)
    H = nx_kitaev_terms(graph, weight_prob)
    return H

g_triangle = nx_triangle_lattice(3)
assign_directional_triangular_labels(g_triangle,3)

In [ ]:
graph_triangle = nx_triangle_lattice(3)
assign_directional_triangular_labels(graph_triangle, 3)
pos = nx.spring_layout(graph_triangle)
edge_labels = dict([((n1, n2), d['label']) for n1, n2, d in graph_triangle.edges(data=True)]);
nx.draw(graph_triangle, pos)
nx.draw_networkx_edge_labels(graph_triangle, pos,edge_labels = edge_labels);

In [ ]:
lattice_size_directional_triangle = 32
# lattice_size_directional_triangle = 3
directional_triangle_hamiltonian = generate_directional_triangular_hamiltonian(lattice_size_directional_triangle)

timesteps = 1000
required_precision = 1e-16
timestep_of_interest = 1 # sim_time
H_directional_triangle = pyH(directional_triangle_hamiltonian)

print('Estimating Directional Triangle', flush=True)
qsp_circ_directional_triangle = estimate_qsp(H_directional_triangle, timesteps,
                                             required_precision, 'QSP/directional_triangle_circuits/',
                                             hamiltonian_name='directional_triangle')
print('Finished Estimating', flush=True)

# Trotter imeplementations
Now we can do all of this again for second order Suzuki Trotter rather than QSP.  Below we will see how to construct a single trotter step of second order Suzuki-Trotter for these hamiltonians along with a loose upper bound for the number of trotter steps required.  According to the [documentation](https://quantumai.google/reference/python/openfermion/circuits/error_bound) from openfermion the trotter step estimate for the second order Suzuki-Trotter expansion.

In [ ]:
def find_hamiltonian_ordering(of_hamiltonian):
    """
    Function to generate a near optimal term ordering for trotterization of transverse field Ising Models.
    This would need to be modified if there were multi-qubit interactions that were not just ZZ
    """
    # ordering hamiltonian terms by performing edge coloring to make optimal trotter ordering
    # assuming that any 2 body interactions are ZZ
    sorted_terms = sorted(list(of_hamiltonian.terms.keys()))
    
    # Z and X get translated to 90 and 88 respectively, multiplying by 100 
    # ensures interacting term weight is considered
    sorted_terms.sort(key=lambda x: len(x) * 100 + ord(x[0][1]))
    
    one_body_terms_ordered = list(filter(lambda x: len(x) == 1, sorted_terms))
    two_body_terms = list(filter(lambda x: len(x) == 2, sorted_terms))
    
    # assigning edge colorings to order two body terms
    graph = Graph()
    for term in two_body_terms:
        edge = (term[0][0], term[1][0])
        graph.add_edge(*edge)
    edge_coloring = nx.greedy_color(nx.line_graph(graph))
    nx.set_edge_attributes(graph, edge_coloring, 'color')
    for (i, term) in enumerate(two_body_terms):
        n1, n2 = (term[0][0], term[1][0])
        color = graph.edges[n1, n2]['color']
        term = (*term, color)
        two_body_terms[i] = term
    
    two_body_terms.sort(key=lambda x: x[2])
    two_body_terms_ordered = list()
    for (i, term) in enumerate(two_body_terms):
        two_body_terms_ordered.append((term[0], term[1]))
    return one_body_terms_ordered + two_body_terms_ordered

In [ ]:
def estimate_trotter(openfermion_hamiltonian,
                     timesteps, 
                     energy_precision, 
                     outdir, 
                     hamiltonian_name="hamiltonian", 
                     write_circuits=False):
    t0 = time.perf_counter()
    bounded_error = error_bound(list(openfermion_hamiltonian.get_operators()),tight=False)
    nsteps = trotter_steps_required(trotter_error_bound = bounded_error,
                                                time = timesteps, 
                                                energy_precision = energy_precision)
    print(f'we require {nsteps} trotter steps')
    t1 = time.perf_counter()
    elapsed = t1 - t0
    print("  Time to estimate Number of steps required: " + str(elapsed) + " seconds")
    
    t0 = time.perf_counter()
    term_ordering = find_hamiltonian_ordering(openfermion_hamiltonian)
    t1 = time.perf_counter()
    elapsed = t1 - t0
    print("  Time to find term ordering: " + str(elapsed) + " seconds")
    
    t0 = time.perf_counter()
    trotter_circuit_of = trotterize_exp_qubop_to_qasm(openfermion_hamiltonian, trotter_order=2, evolution_time=timesteps/nsteps, term_ordering=term_ordering)
    t1 = time.perf_counter()
    elapsed = t1 - t0
    print("  Time to generate trotter circuit from openfermion: " + str(elapsed) + " seconds")
    
    qasm_str_trotter = open_fermion_to_qasm(count_qubits(openfermion_hamiltonian), trotter_circuit_of)
    trotter_circuit_qasm = qasm_import.circuit_from_qasm(qasm_str_trotter)
    
    t0 = time.perf_counter()
    cpt_trotter = clifford_plus_t_direct_transform(trotter_circuit_qasm)
    t1 = time.perf_counter()
    elapsed = t1 - t0
    print("  Time to decompose trotter to Clifford + T: " + str(elapsed) + " seconds")
    
    #writing the the higher level trotter circuit to a file as well as the clifford + T circuit
    if not os.path.exists(outdir):
        os.makedirs(outdir)
    
    if write_circuits:
        outfile_qasm_decomposed = outdir + "trotter_circuit_" + hamiltonian_name + ".qasm" 
        outfile_qasm_cpt = outdir + "trotter_cpt_" + hamiltonian_name + ".qasm"
        with open(outfile_qasm_decomposed, 'w') as f:
            print_to_openqasm(f, trotter_circuit_qasm, qubits=trotter_circuit_qasm.all_qubits())
        with open(outfile_qasm_cpt, 'w') as f:
            print_to_openqasm(f, cpt_trotter, qubits=cpt_trotter.all_qubits())
    
    t0 = time.perf_counter()
    outfile_data = f'{outdir}trotter_{hamiltonian_name}.json'
    trotter_estimate = gen_resource_estimate(cpt_trotter, trotter_steps=nsteps)
    re_as_json(trotter_estimate, [], outfile_data)
    
    t1 = time.perf_counter()
    elapsed = t1 - t0
    print("  Time to enumerate resource estimates: " + str(elapsed) + " seconds")
    
    return cpt_trotter

In [ ]:
# Translating the hamiltonians from above into a form usable by openfermion
openfermion_hamiltonian_square = pyliqtr_hamiltonian_to_openfermion_qubit_operator(H_square)
openfermion_hamiltonian_triangle = pyliqtr_hamiltonian_to_openfermion_qubit_operator(H_triangle)
openfermion_hamiltonian_cube = pyliqtr_hamiltonian_to_openfermion_qubit_operator(H_cube)
openfermion_hamiltonian_kitaev = pyliqtr_hamiltonian_to_openfermion_qubit_operator(H_kitaev)
openfermion_hamiltonian_directional_triangle = pyliqtr_hamiltonian_to_openfermion_qubit_operator(H_directional_triangle)

## Estimates
Trotterizing the Hamiltonians and writing estimates to files

In [ ]:
# defining precision required for the trotterized circuit
energy_precision = 1e-6
timesteps=1000

In [ ]:
print('Estimating Square', flush=True)
cpt_trotter_square = estimate_trotter(openfermion_hamiltonian_square,
                                      timesteps, energy_precision,
                                      'Trotter/square_circuits/',
                                      hamiltonian_name='square')
print('Estimating Triangle', flush=True)
cpt_trotter_triangle = estimate_trotter(openfermion_hamiltonian_triangle,
                                        timesteps, energy_precision,
                                        'Trotter/triangle_circuits/',
                                        hamiltonian_name='triangle')
print('Estimating Cube', flush=True)
cpt_trotter_cube = estimate_trotter(openfermion_hamiltonian_cube,
                                    timesteps, energy_precision,
                                    'Trotter/cube_circuits/',
                                    hamiltonian_name='cube')
print('Estimating Kitaev', flush=True)
cpt_trotter_kitaev = estimate_trotter(openfermion_hamiltonian_kitaev,
                                      timesteps, energy_precision,
                                      'Trotter/kitaev_circuits/',
                                      hamiltonian_name='kitaev')
print('Estimating Directional Triangle', flush=True)
cpt_trotter_directional_triangle = estimate_trotter(openfermion_hamiltonian_directional_triangle, 
                                                    timesteps, energy_precision, 'Trotter/directional_triangle_circuits/', 
                                                    hamiltonian_name='directional_triangle')
print('Finished with estimates', flush=True)

In [ ]:
figdir = 'Trotter/Figures/'
widthdir = 'Trotter/Widths/'
if not os.path.exists(figdir):
    os.makedirs(figdir)
if not os.path.exists(widthdir):
    os.makedirs(widthdir)

In [ ]:
plot_histogram(
    cpt_trotter_square,
    'Trotter Square',
    figdir,
    widthdir
)

In [ ]:
plot_histogram(
    cpt_trotter_triangle,
    'Trotter Triangle',
    figdir,
    widthdir
)

In [ ]:
plot_histogram(
    cpt_trotter_cube,
    'Trotter Cube',
    figdir,
    widthdir
)

In [ ]:
plot_histogram(
    cpt_trotter_kitaev,
    'Trotter Kitaev',
    figdir,
    widthdir
)

In [ ]:
plot_histogram(
    cpt_trotter_directional_triangle,
    'Trotter Directional Triangle',
    figdir,
    widthdir
)